# Federated AIC-Based Feature Selection using GLM with Rhino FCP
This notebook demonstrates how to run a Federated Generalized Linear Model (GLM) using the Rhino FCP SDK and perform AIC-based feature selection in a federated setting. The model is trained using NVFlare. Our goal is to iteratively eliminate variables while minimizing the Akaike Information Criterion (AIC), which balances model fit and complexity.

#### Prerequisites 
1. Have two datasets imported into FCP with variables on which you want to run the regression (e.g. 'Y', 'X', 'COV1', 'COV2', 'COV3', and 'COV4' in this example)
2. Build a container from the NVFlare-based GLM with AIC code from this example and push it to your ECR repo

### Initialization and Login

In [30]:
import getpass
import json
import numpy as np
import copy

import rhino_health as rh
from rhino_health.lib.endpoints.code_object.code_object_dataclass import (
    CodeObject,
    CodeObjectCreateInput,
    CodeTypes,
    ModelTrainInput
)
from rhino_health.lib.endpoints.endpoint import NameFilterMode

In [31]:
my_username = "my_email@example.com" # Replace this with the email you use to log into Rhino Health

print("Logging In")
session = rh.login(username=my_username, password=getpass.getpass())
print("Logged In")

Logging In
········
Logged In


### Load the project, datasets, and data schema

In [32]:
my_project_name = "my_project"  # Replace this with your project name on Rhino Health
project = session.project.get_project_by_name(my_project_name)

In [33]:
dataset_search_string = "MyDataset"  # Replace this with a string that exists in all of the relevant datasets' names
datasets = project.search_for_datasets_by_name(dataset_search_string, name_filter_mode=NameFilterMode.CONTAINS)
schema = datasets[0].data_schema

# Note: There are multiple ways to retrive datasets using the SDK, this examples relies on the datasets having similar names, such as "MyDataset 1" and "MyDataset 2"

### Create the code object with the desired configuration

In [36]:
my_base_ecr_uri = "my_base_ecr_uri"  # Replace this with your workgroup ecr uri
image_name = "image name"  # Replace the name of the docker image uploaded to your ecr, containing the GLM regression with AIC using nvflare code

#### Create the Code Object

In [37]:
code_object_config = CodeObjectCreateInput(
    name=f'GLM Model with AIC', 
    description="GLM MODEL with AIC FEATURE SELECTION",
    input_data_schema_uids=[schema.uid],
    output_data_schema_uids=[None],
    project_uid= project.uid, 
    code_type=CodeTypes.NVIDIA_FLARE_V2_3, 
    config={"container_image_uri": f"{my_base_ecr_uri}:{image_name}"} 
)

code_object = session.code_object.create_code_object(code_object_config)
print(f"Got code object '{code_object.name}' with uid {code_object.uid}")

Got code object 'AIC NVFLARE 2' with uid 9fd45c5a-d515-4d8e-9b44-6dffdfed3c15


### Create the code object with the desired configuration (including regression formula)

#### Define the config (regression family, optimization method, etc.) in the config file

In [35]:
x_values_source = ["Xb","COV1","COV2","COV3","COV4","COV5","COV6"]
y_values = ["Yb"]
method = "IRLS"
glm_type = "Binomial"

config_fed_client_path = "config/config_fed_client.json" # Replace this with the path to your config client file
with open(config_fed_client_path) as f:
    base_config_fed_client = json.load(f)

initial_config = copy.deepcopy(base_config_fed_client)
args = initial_config['executors'][0]['executor']['args']
args['x_values'] = x_values_source
args['y_values'] = y_values
args['method'] = method
args['glm_type'] = glm_type
    
    
config_fed_server_path = "config/config_fed_server.json"  # Replace this with the path to your config server file
with open(config_fed_server_path) as f:
    config_fed_server_input = json.loads(f.read())
    


### AIC-Based Feature Selection Loop

#### Run Initial Model with All Features

In [ ]:
# Step 1: Run full model with all features
run_params = ModelTrainInput(
    code_object_uid=code_object.uid,
    input_dataset_uids=[dataset.uid for dataset in datasets],
    one_fl_client_per_dataset=True,
    validation_dataset_uids=[],
    validation_datasets_inference_suffix="",
    timeout_seconds=1200,
    config_fed_server=json.dumps(config_fed_server_input),
    config_fed_client=json.dumps(initial_config),
    secrets_fed_client="",
    secrets_fed_server="",
    sync=False,
)

print("Running initial model with all features...")
model_train = session.code_object.train_model(run_params)
code_run_uid = model_train.code_run_uid
run_result = model_train.wait_for_completion(timeout_seconds=1500)

model_output = np.load(session.code_run.get_model_params(code_run_uid), allow_pickle=True)
scalar_value = model_output.item()
aic = scalar_value["aic"]
print(f"Initial AIC with all features: {aic}")

#### Iterate Over Features to Identify Removable Candidates

In [43]:
# Initialize best values
best_x_values = copy.deepcopy(x_values_source)
best_aic = aic
improvement = True

# Track results of all iterations
all_results = {}
iteration = 1

# Step 2: Iteratively remove features and test AIC
while improvement and best_x_values:
    
    aic_results = []

    for var in best_x_values:

        test_x = [x for x in best_x_values if x != var]
        test_config = copy.deepcopy(base_config_fed_client)
        args = test_config['executors'][0]['executor']['args']
        args['x_values'] = test_x
        args['y_values'] = y_values
        args['method'] = method
        args['glm_type'] = glm_type

        run_params = ModelTrainInput(
            code_object_uid=code_object.uid,
            input_dataset_uids=[dataset.uid for dataset in datasets],
            one_fl_client_per_dataset=True,
            validation_dataset_uids=[],
            validation_datasets_inference_suffix="",
            timeout_seconds=1200,
            config_fed_server=json.dumps(config_fed_server_input),
            config_fed_client=json.dumps(test_config),
            secrets_fed_client="",
            secrets_fed_server="",
            sync=False,
        )

        print(f"Testing without: {var}")
        model_train = session.code_object.train_model(run_params)
        code_run_uid = model_train.code_run_uid
        run_result = model_train.wait_for_completion(timeout_seconds=1500)

        model_output = np.load(session.code_run.get_model_params(code_run_uid), allow_pickle=True)
        scalar_value = model_output.item()
        aic = scalar_value["aic"]
        print(f"AIC without {var}: {aic}")

        aic_results.append((var, aic))

    # Save current iteration results
    all_results[f"iteration_{iteration}"] = {
        "all_results": aic_results,
        "best_aic": min(aic_results, key=lambda x: x[1])
    }

    best_candidate = min(aic_results, key=lambda x: x[1])
    print(f"Best candidate for removal: {best_candidate}")

    if best_candidate[1] < best_aic:
        best_aic = best_candidate[1]
        best_x_values.remove(best_candidate[0])
        print(f"Removed {best_candidate[0]}, New best AIC: {best_aic}")
    else:
        print("No further improvement in AIC.")
        improvement = False

    iteration += 1


print("Final selected variables:")
print(best_x_values)

# Optional: pretty print all_results at the end
# import pprint
# pprint.pprint(all_results)


Running initial model with all features...
Waiting for code run to complete (0 hours 0 minutes and 2 seconds)
Waiting for code run to complete (0 hours 0 minutes and 13 seconds)
Waiting for code run to complete (0 hours 0 minutes and 25 seconds)
Waiting for code run to complete (0 hours 0 minutes and 36 seconds)
Done.
Initial AIC with all features: 8131.333277221753
Testing without: AGE_CATEG
Waiting for code run to complete (0 hours 0 minutes and a second)
Waiting for code run to complete (0 hours 0 minutes and 13 seconds)
Waiting for code run to complete (0 hours 0 minutes and 25 seconds)
Done.
AIC without AGE_CATEG: 8129.28888875391
Testing without: IS_MALE
Waiting for code run to complete (0 hours 0 minutes and a second)
Waiting for code run to complete (0 hours 0 minutes and 13 seconds)
Waiting for code run to complete (0 hours 0 minutes and 24 seconds)
Waiting for code run to complete (0 hours 0 minutes and 36 seconds)
Done.
AIC without IS_MALE: 8131.847761219306
Testing without:

Waiting for code run to complete (0 hours 17 minutes and 35 seconds)
Waiting for code run to complete (0 hours 17 minutes and 47 seconds)
Waiting for code run to complete (0 hours 17 minutes and 59 seconds)
Done.
AIC without MI: 8129.448728666633
Testing without: AF
Waiting for code run to complete (0 hours 0 minutes and a second)
Waiting for code run to complete (0 hours 0 minutes and 13 seconds)
Waiting for code run to complete (0 hours 0 minutes and 24 seconds)
Waiting for code run to complete (0 hours 0 minutes and 36 seconds)
Done.
AIC without AF: 8138.871902014704
Testing without: CHF
Waiting for code run to complete (0 hours 0 minutes and a second)
Waiting for code run to complete (0 hours 0 minutes and 13 seconds)
Waiting for code run to complete (0 hours 0 minutes and 25 seconds)
Done.
AIC without CHF: 8129.548702648154
Testing without: IHD_NO_MI
Waiting for code run to complete (0 hours 0 minutes and 2 seconds)
Waiting for code run to complete (0 hours 0 minutes and 13 second

Waiting for code run to complete (0 hours 0 minutes and 26 seconds)
Waiting for code run to complete (0 hours 0 minutes and 37 seconds)
Waiting for code run to complete (0 hours 0 minutes and 48 seconds)
Waiting for code run to complete (0 hours a minute and 0 seconds)
Waiting for code run to complete (0 hours a minute and 11 seconds)
Waiting for code run to complete (0 hours a minute and 23 seconds)
Waiting for code run to complete (0 hours a minute and 35 seconds)
Waiting for code run to complete (0 hours a minute and 46 seconds)
Waiting for code run to complete (0 hours a minute and 58 seconds)
Waiting for code run to complete (0 hours 2 minutes and 9 seconds)
Waiting for code run to complete (0 hours 2 minutes and 21 seconds)
Waiting for code run to complete (0 hours 2 minutes and 32 seconds)
Waiting for code run to complete (0 hours 2 minutes and 43 seconds)
Waiting for code run to complete (0 hours 2 minutes and 55 seconds)
Waiting for code run to complete (0 hours 3 minutes and 

Done.
AIC without CANCER: 8130.106711939444
Testing without: HYPOTHYR
Waiting for code run to complete (0 hours 0 minutes and 2 seconds)
Waiting for code run to complete (0 hours 0 minutes and 13 seconds)
Waiting for code run to complete (0 hours 0 minutes and 25 seconds)
Waiting for code run to complete (0 hours 0 minutes and 36 seconds)
Done.
AIC without HYPOTHYR: 8129.372025954311
Testing without: HYPERTHYR
Waiting for code run to complete (0 hours 0 minutes and a second)
Waiting for code run to complete (0 hours 0 minutes and 13 seconds)
Waiting for code run to complete (0 hours 0 minutes and 24 seconds)
Waiting for code run to complete (0 hours 0 minutes and 36 seconds)
Done.
AIC without HYPERTHYR: 8129.577804913869
Testing without: THYRODIT
Waiting for code run to complete (0 hours 0 minutes and a second)
Waiting for code run to complete (0 hours 0 minutes and 13 seconds)
Waiting for code run to complete (0 hours 0 minutes and 25 seconds)
Waiting for code run to complete (0 hours

Waiting for code run to complete (0 hours 16 minutes and 2 seconds)
Waiting for code run to complete (0 hours 16 minutes and 13 seconds)
Waiting for code run to complete (0 hours 16 minutes and 24 seconds)
Waiting for code run to complete (0 hours 16 minutes and 36 seconds)
Waiting for code run to complete (0 hours 16 minutes and 48 seconds)
Waiting for code run to complete (0 hours 17 minutes and 0 seconds)
Waiting for code run to complete (0 hours 17 minutes and 12 seconds)
Waiting for code run to complete (0 hours 17 minutes and 23 seconds)
Waiting for code run to complete (0 hours 17 minutes and 35 seconds)
Waiting for code run to complete (0 hours 17 minutes and 46 seconds)
Waiting for code run to complete (0 hours 17 minutes and 58 seconds)
Waiting for code run to complete (0 hours 18 minutes and 9 seconds)
Waiting for code run to complete (0 hours 18 minutes and 21 seconds)
Waiting for code run to complete (0 hours 18 minutes and 32 seconds)
Done.
AIC without PREV_PERSONALITY_DI

...

Waiting for code run to complete (0 hours 0 minutes and 13 seconds)
Waiting for code run to complete (0 hours 0 minutes and 25 seconds)
Waiting for code run to complete (0 hours 0 minutes and 37 seconds)
Done.
AIC without IHD_NO_MI: 8118.5772890203
Testing without: PVD
Waiting for code run to complete (0 hours 0 minutes and 2 seconds)
Waiting for code run to complete (0 hours 0 minutes and 13 seconds)
Waiting for code run to complete (0 hours 0 minutes and 25 seconds)
Waiting for code run to complete (0 hours 0 minutes and 36 seconds)
Done.
AIC without PVD: 8118.424677462934
Testing without: HYPERTENSIO
Waiting for code run to complete (0 hours 0 minutes and 2 seconds)
Waiting for code run to complete (0 hours 0 minutes and 13 seconds)
Waiting for code run to complete (0 hours 0 minutes and 25 seconds)
Waiting for code run to complete (0 hours 0 minutes and 37 seconds)
Done.
AIC without HYPERTENSIO: 8118.399342387326
Testing without: DM
Waiting for code run to complete (0 hours 0 minut

Done.
AIC without CKD: 8119.416113857323
Testing without: IBD
Waiting for code run to complete (0 hours 0 minutes and 2 seconds)
Waiting for code run to complete (0 hours 0 minutes and 13 seconds)
Waiting for code run to complete (0 hours 0 minutes and 25 seconds)
Waiting for code run to complete (0 hours 0 minutes and 36 seconds)
Waiting for code run to complete (0 hours 0 minutes and 48 seconds)
Waiting for code run to complete (0 hours a minute and 0 seconds)
Waiting for code run to complete (0 hours a minute and 11 seconds)
Waiting for code run to complete (0 hours a minute and 22 seconds)
Waiting for code run to complete (0 hours a minute and 34 seconds)
Waiting for code run to complete (0 hours a minute and 45 seconds)
Waiting for code run to complete (0 hours a minute and 57 seconds)
Waiting for code run to complete (0 hours 2 minutes and 9 seconds)
Waiting for code run to complete (0 hours 2 minutes and 20 seconds)
Waiting for code run to complete (0 hours 2 minutes and 32 seco

Waiting for code run to complete (0 hours 0 minutes and 24 seconds)
Waiting for code run to complete (0 hours 0 minutes and 36 seconds)
Done.
AIC without HYPERTHYR: 8118.65722279256
Testing without: THYRODIT
Waiting for code run to complete (0 hours 0 minutes and a second)
Waiting for code run to complete (0 hours 0 minutes and 13 seconds)
Waiting for code run to complete (0 hours 0 minutes and 25 seconds)
Waiting for code run to complete (0 hours 0 minutes and 36 seconds)
Done.
AIC without THYRODIT: 8118.947484371643
Testing without: PREV_DEP_EPI_C
Waiting for code run to complete (0 hours 0 minutes and 2 seconds)
Waiting for code run to complete (0 hours 0 minutes and 13 seconds)
Waiting for code run to complete (0 hours 0 minutes and 25 seconds)
Waiting for code run to complete (0 hours 0 minutes and 36 seconds)
Done.
AIC without PREV_DEP_EPI_C: 8116.935774729716
Testing without: PREV_ANXIETY_DISORDER
Waiting for code run to complete (0 hours 0 minutes and 2 seconds)
Waiting for cod

Waiting for code run to complete (0 hours 8 minutes and 32 seconds)
Waiting for code run to complete (0 hours 8 minutes and 44 seconds)
Waiting for code run to complete (0 hours 8 minutes and 55 seconds)
Waiting for code run to complete (0 hours 9 minutes and 7 seconds)
Waiting for code run to complete (0 hours 9 minutes and 18 seconds)
Waiting for code run to complete (0 hours 9 minutes and 30 seconds)
Waiting for code run to complete (0 hours 9 minutes and 41 seconds)
Waiting for code run to complete (0 hours 9 minutes and 53 seconds)
Waiting for code run to complete (0 hours 10 minutes and 5 seconds)
Waiting for code run to complete (0 hours 10 minutes and 16 seconds)
Waiting for code run to complete (0 hours 10 minutes and 28 seconds)
Waiting for code run to complete (0 hours 10 minutes and 39 seconds)
Waiting for code run to complete (0 hours 10 minutes and 51 seconds)
Waiting for code run to complete (0 hours 11 minutes and 2 seconds)
Waiting for code run to complete (0 hours 11 

Waiting for code run to complete (0 hours 0 minutes and 25 seconds)
Waiting for code run to complete (0 hours 0 minutes and 36 seconds)
Done.
AIC without PREV_NDRI: 8122.527865206188
Testing without: PREV_NMDA
Waiting for code run to complete (0 hours 0 minutes and 2 seconds)
Waiting for code run to complete (0 hours 0 minutes and 13 seconds)
Waiting for code run to complete (0 hours 0 minutes and 25 seconds)
Waiting for code run to complete (0 hours 0 minutes and 36 seconds)
Done.
AIC without PREV_NMDA: 8123.817527098813
Testing without: PREV_NaRI
Waiting for code run to complete (0 hours 0 minutes and a second)
Waiting for code run to complete (0 hours 0 minutes and 13 seconds)
Waiting for code run to complete (0 hours 0 minutes and 25 seconds)
Waiting for code run to complete (0 hours 0 minutes and 36 seconds)
Done.
AIC without PREV_NaRI: 8121.745767192138
Testing without: PREV_SARI
Waiting for code run to complete (0 hours 0 minutes and 2 seconds)
Waiting for code run to complete (